<a href="https://colab.research.google.com/github/ylchin/AutoAWQ/blob/main/quantization_GPTQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/AutoGPTQ/

/content/drive/MyDrive/AutoGPTQ


In [ ]:
from transformers import AutoTokenizer, TextGenerationPipeline
from auto_gptq.modeling import AutoGPTQForCausalLM, BaseQuantizeConfig

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `dso` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `dso`


Downloads model from huggingface hub and cache it to local disk, then load into CPU memory

In [ ]:
pretrained_model_name = "meta-llama/Llama-2-8B"
quantize_config = BaseQuantizeConfig(
    bits=4, # quantize model to 4 bits
    group_size=128, # recommended to set value to 128
    desc_act=False, # set to False can significantly speed up inference but perplexity may be slightly worse
    )

# load un-quantized model, by default, the model will always be loaded into CPU memory
model = AutoGPTQForCausalLM.from_pretrained(pretrained_model_name, quantize_config)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

prepare examples(a list of dict with only two keys, 'input_ids' and 'attention_mask') to guide quantization. Here we use only one text to simplify the code, but you should be noticed that the more examples used, the better(most likely) the quantized model.

In [ ]:
examples = [
    tokenizer(
        "auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."
    )
]

In [ ]:
model.quantize(examples)

INFO - Start quantizing layer 1/16
INFO - Quantizing self_attn.k_proj in layer 1/16...
INFO - Quantizing self_attn.v_proj in layer 1/16...
INFO - Quantizing self_attn.q_proj in layer 1/16...
INFO - Quantizing self_attn.o_proj in layer 1/16...
INFO - Quantizing mlp.up_proj in layer 1/16...
INFO - Quantizing mlp.gate_proj in layer 1/16...
INFO - Quantizing mlp.down_proj in layer 1/16...
INFO - Start quantizing layer 2/16
INFO - Quantizing self_attn.k_proj in layer 2/16...
INFO - Quantizing self_attn.v_proj in layer 2/16...
INFO - Quantizing self_attn.q_proj in layer 2/16...
INFO - Quantizing self_attn.o_proj in layer 2/16...
INFO - Quantizing mlp.up_proj in layer 2/16...
INFO - Quantizing mlp.gate_proj in layer 2/16...
INFO - Quantizing mlp.down_proj in layer 2/16...
INFO - Start quantizing layer 3/16
INFO - Quantizing self_attn.k_proj in layer 3/16...
INFO - Quantizing self_attn.v_proj in layer 3/16...
INFO - Quantizing self_attn.q_proj in layer 3/16...
INFO - Quantizing self_attn.o_pro

In [ ]:
quantized_model_dir = "Llama-3.1-1B-4bit-128g"
model.save_quantized(quantized_model_dir)  # save quantized model using safetensors
tokenizer.save_pretrained(quantized_model_dir)

In [ ]:
# load quantized model to the first GPU
model = AutoGPTQForCausalLM.from_quantized(quantized_model_dir)

WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
INFO - The layer lm_head is not quantized.


In [ ]:
print(tokenizer.decode(model.generate(**tokenizer("auto_gptq is", return_tensors="pt").to(model.device))[0]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>auto_gptq is a Python package that implements the GPT-2 model for text generation. It is based on the


In [ ]:
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer)
print(pipeline("auto-gptq is")[0]["generated_text"])

Device set to use cuda:0
The model 'LlamaGPTQForCausalLM' is not supported for . Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GP

auto-gptq is a tool which can be used to generate a quantum circuit as a black-box quantum computer.
The quantum circuit is a program that takes a list of quantum gates and applies them to an input quantum state. The algorithm that executes this program is the quantum circuit.
The input is a list of quantum gates and the output is a quantum circuit.
auto-gptq is a tool which can be used to generate a quantum circuit as a black-box quantum computer. The quantum circuit is a program that takes a list of quantum gates and applies them to an input quantum state. The algorithm that executes this program is the quantum circuit.
The input is a list of quantum gates and the output is a quantum circuit. The quantum circuit is a program that takes a list of quantum gates and applies them to an input quantum state. The algorithm that executes this program is the quantum circuit.
